Data from: https://www.kaggle.com/datasets/undersc0re/predict-the-churn-risk-rate

In [53]:
import pandas as pd
import numpy as np
import seaborn as sns

from sklearn.model_selection import train_test_split


In [54]:
df = pd.read_csv("data/churn.csv")
df.head(10)

,Unnamed: 0,age,gender,security_no,region_category,membership_category,joining_date,joined_through_referral,referral_id,preferred_offer_types,...,avg_time_spent,avg_transaction_value,avg_frequency_login_days,points_in_wallet,used_special_discount,offer_application_preference,past_complaint,complaint_status,feedback,churn_risk_score
0,0,18,F,XW0DQ7H,Village,Platinum Membership,2017-08-17,No,xxxxxxxx,Gift Vouchers/Coupons,...,300.63,53005.25,17.0,781.75,Yes,Yes,No,Not Applicable,Products always in Stock,0
1,1,32,F,5K0N3X1,City,Premium Membership,2017-08-28,?,CID21329,Gift Vouchers/Coupons,...,306.34,12838.38,10.0,NaN,Yes,No,Yes,Solved,Quality Customer Care,0
2,2,44,F,1F2TCL3,Town,No Membership,2016-11-11,Yes,CID12313,Gift Vouchers/Coupons,...,516.16,21027.00,22.0,500.69,No,Yes,Yes,Solved in Follow-up,Poor Website,1
3,3,37,M,VJGJ33N,City,No Membership,2016-10-29,Yes,CID3793,Gift Vouchers/Coupons,...,53.27,25239.56,6.0,567.66,No,Yes,Yes,Unsolved,Poor Website,1
4,4,31,F,SVZXCWB,City,No Membership,2017-09-12,No,xxxxxxxx,Credit/Debit Card Offers,...,113.13,24483.66,16.0,663.06,No,Yes,Yes,Solved,Poor Website,1
5,5,13,M,PSG1LGF,City,Gold Membership,2016-01-08,No,xxxxxxxx,Gift Vouchers/Coupons,...,433.62,13884.77,24.0,722.27,Yes,No,Yes,Unsolved,No reason specified,0
6,6,21,M,R3CX1EA,Town,Gold Membership,2015-03-19,Yes,CID24708,Gift Vouchers/Coupons,...,55.38,8982.50,28.0,756.21,Yes,No,Yes,Solved in Follow-up,No reason specified,0
7,7,42,M,4UJ1551,NaN,No Membership,2016-07-12,?,CID56614,Credit/Debit Card Offers,...,429.11,44554.82,24.0,568.08,No,Yes,Yes,Unsolved,Poor Product Quality,1
8,8,44,M,0481QNQ,Village,Silver Membership,2016-12-14,No,xxxxxxxx,Without Offers,...,191.07,18362.31,20.0,NaN,Yes,No,Yes,Solved in Follow-up,Poor Customer Service,0
9,9,45,F,ZHP4MCR,Town,No Membership,2016-11-30,No,xxxxxxxx,Gift Vouchers/Coupons,...,97.31,19244.16,28.0,706.23,No,Yes,Yes,No Information Available,Poor Customer Service,1


In [19]:
df["feedback"].value_counts()

Poor Product Quality        6350
No reason specified         6290
Too many ads                6279
Poor Website                6271
Poor Customer Service       6252
Reasonable Price            1417
User Friendly Website       1391
Products always in Stock    1382
Quality Customer Care       1360
Name: feedback, dtype: int64

In [20]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 36992 entries, 0 to 36991
Data columns (total 24 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   Unnamed: 0                    36992 non-null  int64  
 1   age                           36992 non-null  int64  
 2   gender                        36992 non-null  object 
 3   security_no                   36992 non-null  object 
 4   region_category               31564 non-null  object 
 5   membership_category           36992 non-null  object 
 6   joining_date                  36992 non-null  object 
 7   joined_through_referral       36992 non-null  object 
 8   referral_id                   36992 non-null  object 
 9   preferred_offer_types         36704 non-null  object 
 10  medium_of_operation           36992 non-null  object 
 11  internet_option               36992 non-null  object 
 12  last_visit_time               36992 non-null  object 
 13  d

Looking at the above, we can already see there is missing data for the following columns:
region_category, preferred_offer_types, and points_in_wallet. 


In [45]:
df["age"].value_counts()

33    720
42    716
16    716
38    714
30    711
61    709
60    704
57    704
41    699
59    696
55    695
28    694
34    692
47    691
21    689
32    685
17    683
56    682
29    681
48    680
45    680
46    679
58    678
62    677
22    677
26    675
53    675
52    672
64    672
10    670
14    670
49    669
23    669
36    668
44    661
12    661
19    660
51    660
27    659
43    657
63    656
35    656
50    656
11    654
13    654
37    653
15    649
24    645
39    642
20    641
31    636
25    635
18    629
40    628
54    608
Name: age, dtype: int64

In [21]:
df["gender"].value_counts()

F          18490
M          18443
Unknown       59
Name: gender, dtype: int64

In [32]:
df["region_category"].value_counts()

Town       14128
City       12737
Village     4699
Name: region_category, dtype: int64

In [26]:
df["joined_through_referral"].value_counts()

No     15839
Yes    15715
?       5438
Name: joined_through_referral, dtype: int64

In [30]:
df["preferred_offer_types"].value_counts()

Gift Vouchers/Coupons       12349
Credit/Debit Card Offers    12274
Without Offers              12081
Name: preferred_offer_types, dtype: int64

In [33]:
df["internet_option"].value_counts()

Wi-Fi          12413
Mobile_Data    12343
Fiber_Optic    12236
Name: internet_option, dtype: int64

In [34]:
df["last_visit_time"].value_counts()

07:56:41    6
17:11:53    5
22:31:06    5
02:50:54    5
12:30:02    5
           ..
21:48:50    1
07:53:40    1
00:05:51    1
10:48:11    1
01:39:52    1
Name: last_visit_time, Length: 30101, dtype: int64

In [35]:
df["days_since_last_login"].value_counts()

 12     2380
 13     2373
 14     2307
 15     2278
 11     2262
 10     2091
 16     2068
-999    1999
 9      1863
 17     1747
 8      1571
 18     1444
 7      1442
 19     1308
 6      1257
 5      1234
 20     1184
 21     1015
 4       998
 22      895
 3       852
 23      727
 2       613
 24      471
 1       328
 25      203
 26       82
Name: days_since_last_login, dtype: int64

In [36]:
df["avg_time_spent"].value_counts()

34.100000      21
34.710000      20
33.680000      20
34.330000      19
31.490000      18
               ..
1177.998747     1
51.550000       1
265.190000      1
447.990000      1
482.610000      1
Name: avg_time_spent, Length: 25961, dtype: int64

In [37]:
df["avg_transaction_value"].value_counts()

14176.97    2
7282.58     2
30126.02    2
21244.03    2
23142.51    2
           ..
40634.78    1
30596.16    1
3300.82     1
2378.69     1
2189.68     1
Name: avg_transaction_value, Length: 36894, dtype: int64

In [55]:
df["avg_frequency_login_days"].value_counts()

Error                  3522
13.0                   1394
19.0                   1365
8.0                    1361
14.0                   1355
                       ... 
28.191570401129514        1
41.73357294995208         1
-11.515939810499656       1
45.71683637272365         1
27.8399274405269          1
Name: avg_frequency_login_days, Length: 1654, dtype: int64

In [39]:
df["points_in_wallet"].value_counts()

705.070000     9
780.920000     8
771.750000     7
760.540000     7
710.690000     7
              ..
1188.729586    1
827.500000     1
357.996152     1
392.010095     1
197.264414     1
Name: points_in_wallet, Length: 23699, dtype: int64

In [40]:
df["used_special_discount"].value_counts()

Yes    20342
No     16650
Name: used_special_discount, dtype: int64

In [41]:
df["offer_application_preference"].value_counts()

Yes    20440
No     16552
Name: offer_application_preference, dtype: int64

In [42]:
df["past_complaint"].value_counts()

No     18602
Yes    18390
Name: past_complaint, dtype: int64

In [43]:
df["complaint_status"].value_counts()

Not Applicable              18602
Unsolved                     4644
Solved                       4619
Solved in Follow-up          4577
No Information Available     4550
Name: complaint_status, dtype: int64

In [44]:
df["feedback"].value_counts()

Poor Product Quality        6350
No reason specified         6290
Too many ads                6279
Poor Website                6271
Poor Customer Service       6252
Reasonable Price            1417
User Friendly Website       1391
Products always in Stock    1382
Quality Customer Care       1360
Name: feedback, dtype: int64

In [6]:
df["churn_risk_score"].value_counts()

1    20012
0    16980
Name: churn_risk_score, dtype: int64

We are also missing data for gender, joined_through_referral, days_since_last_login (-999 is a placeholder it seems), avg_frequency_login_days (which also has values less than 0), and complaint_status.   

In [67]:
df.corr()

/var/folders/x7/5j65n6x57hq9g5yst5z9k1m00000gn/T/ipykernel_90934/1134722465.py:1: FutureWarning: The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  df.corr()


,Unnamed: 0,age,days_since_last_login,avg_time_spent,avg_transaction_value,points_in_wallet,churn_risk_score
Unnamed: 0,1.000000,-0.008361,0.004424,-0.008343,0.004632,-0.001233,0.002325
age,-0.008361,1.000000,-0.003970,0.001531,-0.000473,0.000205,0.006124
days_since_last_login,0.004424,-0.003970,1.000000,-0.002369,0.003934,0.008989,-0.006228
avg_time_spent,-0.008343,0.001531,-0.002369,1.000000,0.020888,-0.000648,-0.014263
avg_transaction_value,0.004632,-0.000473,0.003934,0.020888,1.000000,0.087950,-0.218012
points_in_wallet,-0.001233,0.000205,0.008989,-0.000648,0.087950,1.000000,-0.307439
churn_risk_score,0.002325,0.006124,-0.006228,-0.014263,-0.218012,-0.307439,1.000000


Not seeing strong correlations between the numerical columns and our target but there's still room to explore further.

In [68]:
from sklearn.model_selection import train_test_split

y = df["churn_risk_score"]
X = df.drop(columns="churn_risk_score")

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = .5, random_state = 42)